# Decision analysis with Julia `DecisionProgramming.jl` using Evans, 1997  --  Claude
Tomás Aragón, Updated 2026-01-21

I mostly replicated R `rdecision` vignette "Elementary decision tree (Evans 1997)": https://cran.r-project.org/web/packages/rdecision/vignettes/DT01-Sumatriptan.html . 

Sources:
- Briggs, Andrew Harvey, Karl Claxton, and Mark Sculpher. Decision Modelling for Health Economic Evaluation. Repr. [d. korr. Ausg. von 2007]. Handbooks in Health Economic Evaluation Series. Oxford Univ. Press, 2011.
- Evans, K. W., J. A. Boan, J. L. Evans, and A. Shuaib. “Economic Evaluation of Oral Sumatriptan Compared with Oral Caffeine/Ergotamine for Migraine.” PharmacoEconomics 12, no. 5 (1997): 565–77. https://doi.org/10.2165/00019053-199712050-00007.

This Julia Jupyter notebook was created in VS Code and contains examples from Evans, 1997. You can run this notebook in VS Code, Positron, or JupyterLab with a Julia kernel. To learn about the Julia language visit https://julialang.org/ .

We will use `DecisionProgramming.jl` Julia package: https://gamma-opt.github.io/DecisionProgramming.jl/dev/ . `DecisionProgramming.jl` is a Julia package for modeling and solving decision analysis problems using influence diagrams.

Preparation: 
- PENDING


Optional readings for background:
- Owens, Douglas K. “Analytic Tools for Public Health Decision Making.” Medical Decision Making: An International Journal of the Society for Medical Decision Making 22, no. 5 Suppl (2002): S3-10. https://doi.org/10.1177/027298902237969.
- Owens, D. K., R. D. Shachter, and R. F. Nease. “Representation and Analysis of Medical Decision Problems with Influence Diagrams.” Medical Decision Making: An International Journal of the Society for Medical Decision Making 17, no. 3 (1997): 241–62. https://doi.org/10.1177/0272989X9701700301.
- Nease, R. F., and D. K. Owens. “Use of Influence Diagrams to Structure Medical Decisions.” Medical Decision Making: An International Journal of the Society for Medical Decision Making 17, no. 3 (1997): 263–75. https://doi.org/10.1177/0272989X9701700302.
- Neapolitan, Richard, Xia Jiang, Daniela P. Ladner, and Bruce Kaplan. “A Primer on Bayesian Decision Analysis With an Application to a Kidney Transplant Decision.” Transplantation 100, no. 3 (2016): 489–96. https://doi.org/10.1097/TP.0000000000001145.

In [1]:
using JuMP, HiGHS
using DecisionProgramming

# ============================================================
# Helper function: builds the chance node structure (shared)
# ============================================================
function build_chance_structure!(diagram)
    add_node!(diagram, DecisionNode("D", [], ["D1", "D2"]))
    add_node!(diagram, ChanceNode("R", ["D"], ["R1", "R2"]))
    add_node!(diagram, ChanceNode("N", ["D", "R"], ["N1", "N2", "NA"]))
    add_node!(diagram, ChanceNode("E", ["R"], ["E1", "E2", "NA"]))
    add_node!(diagram, ChanceNode("H", ["E"], ["H1", "H2", "NA"]))
end

function add_chance_probabilities!(diagram)
    # P(R | D)
    X_R = ProbabilityMatrix(diagram, "R")
    X_R["D1", "R1"] = 0.558;  X_R["D1", "R2"] = 0.442
    X_R["D2", "R1"] = 0.379;  X_R["D2", "R2"] = 0.621
    add_probabilities!(diagram, "R", X_R)

    # P(N | D, R)
    X_N = ProbabilityMatrix(diagram, "N")
    X_N["D1", "R1", "N1"] = 0.594;  X_N["D1", "R1", "N2"] = 0.406;  X_N["D1", "R1", "NA"] = 0.0
    X_N["D1", "R2", "N1"] = 0.0;    X_N["D1", "R2", "N2"] = 0.0;    X_N["D1", "R2", "NA"] = 1.0
    X_N["D2", "R1", "N1"] = 0.703;  X_N["D2", "R1", "N2"] = 0.297;  X_N["D2", "R1", "NA"] = 0.0
    X_N["D2", "R2", "N1"] = 0.0;    X_N["D2", "R2", "N2"] = 0.0;    X_N["D2", "R2", "NA"] = 1.0
    add_probabilities!(diagram, "N", X_N)

    # P(E | R)
    X_E = ProbabilityMatrix(diagram, "E")
    X_E["R1", "E1"] = 0.0;  X_E["R1", "E2"] = 0.0;   X_E["R1", "NA"] = 1.0
    X_E["R2", "E1"] = 0.92; X_E["R2", "E2"] = 0.08;   X_E["R2", "NA"] = 0.0
    add_probabilities!(diagram, "E", X_E)

    # P(H | E)
    X_H = ProbabilityMatrix(diagram, "H")
    X_H["E1", "H1"] = 0.0;   X_H["E1", "H2"] = 0.0;   X_H["E1", "NA"] = 1.0
    X_H["E2", "H1"] = 0.998;  X_H["E2", "H2"] = 0.002;  X_H["E2", "NA"] = 0.0
    X_H["NA", "H1"] = 0.0;   X_H["NA", "H2"] = 0.0;    X_H["NA", "NA"] = 1.0
    add_probabilities!(diagram, "H", X_H)
end

# ============================================================
# DIAGRAM 1: Utility only
# ============================================================
diag_u = InfluenceDiagram()
build_chance_structure!(diag_u)
add_node!(diag_u, ValueNode("U", ["D", "N", "E", "H"]))
generate_arcs!(diag_u)
add_chance_probabilities!(diag_u)

Y_U = UtilityMatrix(diag_u, "U")
Y_U .= 0.0    # ← add this line
# Only assign reachable states (all others default to 0.0)
Y_U["D1", "N1", "NA", "NA"] =  1.0    # A: relief, no recurrence
Y_U["D1", "N2", "NA", "NA"] =  0.9    # B: relief, recurrence
Y_U["D1", "NA", "E1", "NA"] = -0.3    # C: no relief, endures
Y_U["D1", "NA", "E2", "H1"] =  0.1    # D: no relief, ED relief
Y_U["D1", "NA", "E2", "H2"] = -0.3    # E: no relief, hospitalized
Y_U["D2", "N1", "NA", "NA"] =  1.0    # F
Y_U["D2", "N2", "NA", "NA"] =  0.9    # G
Y_U["D2", "NA", "E1", "NA"] = -0.3    # H
Y_U["D2", "NA", "E2", "H1"] =  0.1    # I
Y_U["D2", "NA", "E2", "H2"] = -0.3    # J
add_utilities!(diag_u, "U", Y_U)

# ============================================================
# DIAGRAM 2: Cost only (C1 + C2 + C3 + C4)
# ============================================================
diag_c = InfluenceDiagram()
build_chance_structure!(diag_c)
add_node!(diag_c, ValueNode("C1", ["D"]))
add_node!(diag_c, ValueNode("C2", ["D", "N"]))
add_node!(diag_c, ValueNode("C3", ["E"]))
add_node!(diag_c, ValueNode("C4", ["H"]))
generate_arcs!(diag_c)
add_chance_probabilities!(diag_c)

Y_C1 = UtilityMatrix(diag_c, "C1")
Y_C1["D1"] = 16.10;  Y_C1["D2"] = 1.32

Y_C2 = UtilityMatrix(diag_c, "C2")
Y_C2["D1", "N1"] = 0.0;   Y_C2["D1", "N2"] = 16.10;  Y_C2["D1", "NA"] = 0.0
Y_C2["D2", "N1"] = 0.0;   Y_C2["D2", "N2"] = 1.32;   Y_C2["D2", "NA"] = 0.0

Y_C3 = UtilityMatrix(diag_c, "C3")
Y_C3["E1"] = 0.0;  Y_C3["E2"] = 63.16;  Y_C3["NA"] = 0.0

Y_C4 = UtilityMatrix(diag_c, "C4")
Y_C4["H1"] = 0.0;  Y_C4["H2"] = 1093.0;  Y_C4["NA"] = 0.0

add_utilities!(diag_c, "C1", Y_C1)
add_utilities!(diag_c, "C2", Y_C2)
add_utilities!(diag_c, "C3", Y_C3)
add_utilities!(diag_c, "C4", Y_C4)

# ============================================================
# Evaluate each strategy
# ============================================================
optimizer = optimizer_with_attributes(() -> HiGHS.Optimizer())

function evaluate_strategy(diagram, decision_state::Int)
    model, z, variables = generate_model(diagram, model_type="RJT")
    d_vars = z["D"].z    # access the underlying VariableRef array
    for i in 1:2
        @constraint(model, d_vars[i] == (i == decision_state ? 1 : 0))
    end
    set_optimizer(model, optimizer)
    optimize!(model)
    Z = DecisionStrategy(diagram, z)
    S_prob = StateProbabilities(diagram, Z)
    U_dist = UtilityDistribution(diagram, Z)
    return Z, S_prob, U_dist
end

# --- Expected Utility per strategy ---
_, _, U_dist_D1 = evaluate_strategy(diag_u, 1)
_, _, U_dist_D2 = evaluate_strategy(diag_u, 2)
EU_D1 = U_dist_D1.u' * U_dist_D1.p    # or use print_statistics
EU_D2 = U_dist_D2.u' * U_dist_D2.p

# --- Expected Cost per strategy ---
_, _, C_dist_D1 = evaluate_strategy(diag_c, 1)
_, _, C_dist_D2 = evaluate_strategy(diag_c, 2)
EC_D1 = C_dist_D1.u' * C_dist_D1.p
EC_D2 = C_dist_D2.u' * C_dist_D2.p

# ============================================================
# Cost-Effectiveness Analysis
# ============================================================
println("=== Cost-Effectiveness Analysis ===")
println("Strategy         E[Cost]     E[Utility]")
println("Caffeine/Ergot   $(round(EC_D2, digits=4))    $(round(EU_D2, digits=4))")
println("Sumatriptan      $(round(EC_D1, digits=4))   $(round(EU_D1, digits=4))")
println()

ΔC = EC_D1 - EC_D2
ΔU = EU_D1 - EU_D2
println("Incremental Cost:    $(round(ΔC, digits=4))")
println("Incremental Utility: $(round(ΔU, digits=4))")

# ICER per QALY (24-hour time horizon → annualize by multiplying by 365)
ICER_per_QALY = (ΔC / ΔU) * 365
println("ICER: \$$(round(ICER_per_QALY, digits=0)) Can/QALY")

# Expected R reference values:
# E[C|Caffeine]    = 4.7150,  E[U|Caffeine]    = 0.2013
# E[C|Sumatriptan] = 22.0581, E[U|Sumatriptan] = 0.4169
# ICER ≈ $29,363 Can/QALY

Running HiGHS 1.12.0 (git hash: 755a8e027): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 189 rows; 147 cols; 579 nonzeros; 2 integer variables (2 binary)
Coefficient ranges:
  Matrix  [2e-03, 1e+00]
  Cost    [1e-01, 1e+00]
  Bound   [1e+00, 1e+00]
  RHS     [1e+00, 1e+00]
Presolving model
24 rows, 20 cols, 58 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-189); columns 0(-147); nonzeros 0(-579) - Reduced to empty
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u => Trivial upper; z => Trivial zero

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Lea

Results from this notebook

```julia
=== Cost-Effectiveness Analysis ===
Strategy         E[Cost]     E[Utility]
Caffeine/Ergot    4.715      0.2013
Sumatriptan      22.0581     0.4169

Incremental Cost:    17.3431
Incremental Utility: 0.2156
ICER: $29363.0 Can/QALY
```

Results from `NB_R_rdecision_Evans1997.ipynb`

```julia
Run Migraine.... TREATMENT ..d01. Probability      Cost Benefit   Utility          QALY
1   1        Caffeine/Ergot (e02)           1  4.714972       0 0.2012760  0.0005510635
2   1           Sumatriptan (e01)           1 22.058057       0 0.4168609  0.0011413030
```